In [ ]:
import torch
import sys
import gc

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, WrapperConfig, SMOL_LM_135M, LLAMA_3_2_3B

In [47]:
cfg = WrapperConfig(
    model_id_or_path=LLAMA_3_2_3B,
    max_samples=1000,
    data_module_choice="conversation",
    notebook_mode=True,
    tuning_mode="sft",
    special_tokens=["<thought>", "</thought>", "<solution>", "</solution>"],
    input_dataset_name="codecontests_cot_sft_formatted_thoughts_conversations.parquet",
    train_on_inputs=True
)

wrapper = TrainerWrapper(cfg)

2025-01-06 21:18:48.041 | INFO     | trl_wrapper.trainer_wrapper:__init__:178 - Adding special tokens: ['<thought>', '</thought>', '<solution>', '</solution>']


In [ ]:
wrapper.init_model()

In [ ]:
gc.collect()
torch.cuda.empty_cache()
! echo $CUDA_VISIBLE_DEVICES
! nvidia-smi

In [54]:
wrapper.init_data_module()

2025-01-06 21:21:05.852 | INFO     | model.utils:__init__:110 - Cache dir: ../dataset_caches/conversation_data_module
2025-01-06 21:21:05.854 | INFO     | model.utils:setup:128 - Loading dataset for stage fit
2025-01-06 21:21:05.854 | INFO     | model.utils:setup:131 - Processing dataset for stage fit, workers: 1, cache dir ../dataset_caches/conversation_data_module, using cache: False
2025-01-06 21:21:05.877 | INFO     | dataset.conversation:load_dataset:23 - Loading dataset
2025-01-06 21:21:05.897 | INFO     | model.utils:setup:144 - Train dataset size: 2565 Val dataset size: 286
Map: 100%|██████████| 286/286 [00:00<00:00, 322.39 examples/s]


In [55]:
# Tokenizer dev
first_batch = wrapper.data_module.train_dataset[0]
# tokenized = wrapper.data_module.tokenize_conversation([first_batch["conversation"]])
for k in first_batch.keys():
    first_batch[k] = torch.as_tensor(first_batch[k], dtype=torch.long)
wrapper.data_module.visualize_sample(first_batch)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 06 Jan 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Solve the following programming problem in Python.
Roma is playing a new expansion for his favorite game World of Darkraft. He made a new character and is going for 
his first grind.

Roma has a choice to buy exactly one of n different weapons and exactly one of m different armor sets. Weapon i has
attack modifier a_i and is worth ca_i coins, and armor set j has defense modifier b_j and is worth cb_j coins.

After choosing his equipment Roma can proceed to defeat some monsters. There are p monsters he can try to defeat. 
Monster k has defense x_k, attack y_k and possesses z_k coins. Roma can defeat a monster if his weapon's attack 
modifier is larger than the monster's defense, and his armor set's defense modifier is larger than the monster's 
attack. That is, a monster k can be defeated with a weapon i and an armor set j if a_i > x_k and b_j > y_k. After 
defeating the monster, Roma takes all the coins from them. During the grind, Roma can defeat as many monsters as he
likes. Monsters do not respawn, thus each monster can be defeated at most one.

Thanks to Roma's excessive donations, we can assume that he has an infinite amount of in-game currency and can 
afford any of the weapons and armor sets. Still, he wants to maximize the profit of the grind. The profit is 
defined as the total coins obtained from all defeated monsters minus the cost of his equipment. Note that Roma must
purchase a weapon and an armor set even if he can not cover their cost with obtained coins.

Help Roma find the maximum profit of the grind.

Input

The first line contains three integers n, m, and p (1 ≤ n, m, p ≤ 2 ��� 10^5) — the number of available weapons, 
armor sets and monsters respectively.

The following n lines describe available weapons. The i-th of these lines contains two integers a_i and ca_i (1 ≤ 
a_i ≤ 10^6, 1 ≤ ca_i ≤ 10^9) — the attack modifier and the cost of the weapon i.

The following m lines describe available armor sets. The j-th of these lines contains two integers b_j and cb_j (1 
≤ b_j ≤ 10^6, 1 ≤ cb_j ≤ 10^9) — the defense modifier and the cost of the armor set j.

The following p lines describe monsters. The k-th of these lines contains three integers x_k, y_k, z_k (1 ≤ x_k, 
y_k ≤ 10^6, 1 ≤ z_k ≤ 10^3) — defense, attack and the number of coins of the monster k.

Output

Print a single integer — the maximum profit of the grind.

Example

Input


2 3 3
2 3
4 7
2 4
3 2
5 11
1 2 4
2 1 6
3 4 6


Output


1<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<thought>Roma can choose one weapon and one armor set. Each weapon has an attack modifier and a cost, while each 
armor set has a defense modifier and a cost. Monsters have their own defense and attack values, and Roma can defeat
a monster if his weapon's attack modifier is greater than the monster's defense and his armor's defense modifier is
greater than the monster's attack. The profit is calculated as the total coins obtained from defeated monsters 
minus the total cost of the chosen weapon and armor.</thought>
<thought>We need to read the number of weapons, armor sets, and monsters, followed by their respective 
attributes.</thought>
<thought>For each monster, we need to determine which weapons and armor sets can defeat it. This means we need to 
keep track of the total coins from all monsters that can be defeated with the chosen equipment.</thought>
<thought>For each valid weapon and armor combination, we need to calculate the total cost and subtract it from the 
total coins obtained from defeated monsters to find the profit.</thought>
<thought>We need to iterate through all possible combinations of weapons and armor sets, calculate the profit for 
each combination, and keep track of the maximum profit found.</thought>

<solution>def max_profit(n, m, p, weapons, armors, monsters):
    # Ste

In [51]:
mock_labels = first_batch["labels"].clone().detach()
mock_labels = torch.where(mock_labels == -100, wrapper.data_module.tokenizer.pad_token_id, mock_labels)
mock_ids = first_batch["input_ids"].clone().detach()
print(wrapper.data_module.tokenizer.decode(mock_labels))

In [ ]:
wrapper.init_trainer()

In [ ]:
wrapper.train()